# Sandbox
This notebook covers the main steps in deploying a model.  Experimentation with features is encouraged.  Can you increase the NDCG reported during model training?

## Setup Client

In [1]:
from ltr.client import OpenSearchClient
client = OpenSearchClient()

http://localhost:9201/_ltr; <OpenSearch([{'host': 'localhost', 'port': 9201}])>


## Step 1 - Create a Feature Set

In [2]:
'''
  TASK:
  Experiment with this featureset, the judgments are all title searches so what features could be useful?
  
  Ideas:
    - Search other fields
    - Phrase matches
    - Fuzzy matches
'''

client.reset_ltr(index='tmdb')

config = {
   "featureset": {
        "features": [
            {
                "name": "title_match",
                "params": ["keywords"],
                "template": {
                    "match": {
                        "title": "{{keywords}}"
                    }
                }
            }
        ]
    }
}

client.create_featureset(index='tmdb', name='sandbox', ftr_config=config)

Removed Default LTR feature store [Status: 200]
Initialize Default LTR feature store [Status: 200]
Create sandbox feature set [Status: 201]


## Step 2 - Log Features for Training

In [3]:
from ltr.log import FeatureLogger
from ltr.judgments import judgments_open
from itertools import groupby

ftr_logger=FeatureLogger(client, index='tmdb', feature_set='sandbox')
with judgments_open('data/title_judgments.txt') as judgment_list:
    for qid, query_judgments in groupby(judgment_list, key=lambda j: j.qid):
        ftr_logger.log_for_qid(judgments=query_judgments, 
                               qid=qid,
                               keywords=judgment_list.keywords(qid))


Recognizing 40 queries


## Step 3 - Train a Model

In [4]:
'''
  TASK:
  Experiment with the leafs and trees variables, how do they affect NGCG?
  Does a high leaf value increase your NDCG?  What could be the potential downfalls?
'''
from ltr.ranklib import train
trainResponse  = train(client,
                  index='tmdb',
                  training_set=ftr_logger.logged,
                  metric2t='NDCG@10',
                  leafs=20,
                  trees=20,
                  featureSet='sandbox',
                  modelName='sandbox')

trainLog = trainResponse.trainingLogs[0]
print()
print("Impact of each feature on the model")
for ftrId, impact in trainLog.impacts.items():
    print("{} - {}".format(client.get_feature_name(config, ftrId), impact))
    
for roundDcg in trainLog.rounds:
    print(roundDcg)
    
print("Train NDCG@10 %s" % trainLog.rounds[-1])

/var/folders/33/jx0mw87156q2hmtrr_r82s7r0000gs/T/RankyMcRankFace.jar already exists
Running java -jar /var/folders/33/jx0mw87156q2hmtrr_r82s7r0000gs/T/RankyMcRankFace.jar -ranker 6 -shrinkage 0.1 -metric2t NDCG@10 -tree 20 -bag 1 -leaf 20 -frate 1.0 -srate 1.0 -train /var/folders/33/jx0mw87156q2hmtrr_r82s7r0000gs/T/training.txt -save data/sandbox_model.txt 
Delete model sandbox: 404
Created Model sandbox [Status: 201]
Model saved

Impact of each feature on the model
title_match - 35493.99538433604
0.9124
0.9124
0.9124
0.9124
0.9124
0.9124
0.9102
0.9073
0.9073
0.9072
0.9075
0.9093
0.9099
0.9099
0.9099
0.9099
0.9099
0.9099
0.9107
0.9116
Train NDCG@10 0.9116


## Search

In [5]:
from ltr import search
search(client, "rambo", modelName='sandbox')

{"size": 5, "query": {"sltr": {"params": {"keywords": "rambo", "keywordsList": ["rambo"]}, "model": "sandbox"}}}
{'size': 5, 'query': {'sltr': {'params': {'keywords': 'rambo', 'keywordsList': ['rambo']}, 'model': 'sandbox'}}}
Rambo 
2.269571 
2008 
['Action', 'Thriller'] 
When governments fail to act on behalf of captive missionaries, ex-Green Beret John James Rambo sets aside his peaceful existence along the Salween River in a war-torn region of Thailand to take action.  Although he's still haunted by violent memories of his time as a U.S. soldier during the Vietnam War, Rambo can hardly turn his back on the aid workers who so desperately need his help. 
---------------------------------------
Rambo III 
1.7274473 
1988 
['Action', 'Adventure', 'Thriller', 'War'] 
Combat has taken its toll on Rambo, but he's finally begun to find inner peace in a monastery. When Rambo's friend and mentor Col. Trautman asks for his help on a top secret mission to Afghanistan, Rambo declines but must re